# Evaluating the I<sub>t,θ</sub> series

## Testing integration of oscillating fcts, I<sub>t,θ</sub>

In [2]:
using DBNUpperBound
using DBNUpperBound.Asymptotics

x = 300.0
y = 0.4
n = 1
t = 0.4
θ = θ_default(y)
basic_limits = ω_pds(x,t,π*n^2,θ,0.0,0.6)
Itθ_tail(0.6, x, y, n, 9; t=6, θ = θ)

7.765970584865725e-16

Divide basic limits, each of which approximately covers one period of $\omega$, into 10ths and check that piecewise quadrature over the finer limits matches that of the cruder.

In [14]:
fine_limits = collect(0.0:(0.1*mean(diff(basic_limits))):0.6)
fine_limits[end] = basic_limits[end]
@show mean(diff(basic_limits))
@show mean(diff(fine_limits))
nothing

mean(diff(basic_limits)) = 0.021098686810575244
mean(diff(fine_limits)) = 0.0021544433715024016


In [8]:
@show pwItθ(x,y,n,9, basic_limits; t=t, θ=θ)
@show pwItθ(x,y,n,9, fine_limits; t=t, θ=θ)
nothing

pwItθ(x, y, n, 9, basic_limits; t=t, θ=θ) = (3.2970043424797623e-7 + 5.9539425957089085e-6im, 1.2121764916169488e-16, 1.951054189094412e-17)
pwItθ(x, y, n, 9, fine_limits; t=t, θ=θ) = (3.2970043424798693e-7 + 5.953942595708896e-6im, 1.0248774374854885e-19, 1.951054189094412e-17)


In [9]:
@show pwItθ(x,y,n,5, basic_limits; t=t, θ=θ)
@show pwItθ(x,y,n,5, fine_limits; t=t, θ=θ)
nothing

pwItθ(x, y, n, 5, basic_limits; t=t, θ=θ) = (6.616237679118809e-7 + 5.924653415049598e-6im, 9.111542247692362e-17, 1.639444318801552e-18)
pwItθ(x, y, n, 5, fine_limits; t=t, θ=θ) = (6.616237679118824e-7 + 5.9246534150495916e-6im, 6.694674107003369e-20, 1.639444318801552e-18)


## Evaluating terms in the series

We have from [Bounding the derivative 2](http://michaelnielsen.org/polymath1/index.php?title=Bounding_the_derivative_of_H_t_-_second_approach)

$$H_t(z) = \frac{1}{2} \sum_{n=1}^\infty 2\pi^2 n^4 I_{t,\theta}(x-(9-y)i, \pi n^2) - 3\pi n^2 I_{t,\theta}(x-(5-y)i, \pi n^2) + 2\pi^2 n^4 \overline{I_{t,\theta}(x-(9+y)i, \pi n^2)} - 3\pi n^2 \overline{I_{t,\theta}(x-(5+y)i, \pi n^2)}$$

Coding a function which will evaluate one term in this series together with error estimates.

In [3]:
function Itθ_term(x,y,t,n,X; θ=θ_default, limits = ω_pds(x,t,π*n^2,θ,0.0,X))
    I9 = pwItθ(x,y,n,9,limits, t=t, θ=θ)
    I5 = pwItθ(x,y,n,5,limits, t=t, θ=θ)
    I9c = pwItθ(x,-y,n,9,limits, t=t, θ=θ)
    I5c = pwItθ(x,-y,n,5,limits, t=t, θ=θ)
    val = 2*π^2*n^4*(I9[1]+I9c[1]') - 3*π*n^2*(I5[1]+I5c[1]')
    integration_error = 2*π^2*n^4*(I9[2]+I9c[2]) + 3*π*n^2*(I5[2]+I5c[2])
    tail_bound = 2*π^2*n^4*(I9[3]+I9c[3]) + 3*π*n^2*(I5[3]+I5c[3])
    return val, integration_error, tail_bound
end

Itθ_term (generic function with 1 method)

In [4]:
Itθ_term(x,y,t,n,0.6; θ=θ)

(-1.4518521362713932e-7 + 8.510239639471211e-10im, 6.707991182416713e-15, 6.446592467831734e-16)

In [11]:
Itθ_term(x,y,t,n,0.6; θ=θ, limits=fine_limits)

(-1.451852136268564e-7 + 8.510239638657927e-10im, 5.517775170572531e-18, 6.446592467831734e-16)

In [11]:
function pwHt(x,y,t,N,X; θ=θ_default(y), limits = ω_pds(x,t,π*N^2,θ,0.0,X))
    N ≥ 1 || error("N should be at least 1")
    val = integration_error = integration_tail = series_bounds = 0.0
    n = 1
    limits = [typeof(X)(u) for u in limits]
    while n ≤ N
        @show n
        @show tmp = Itθ_term(x,y,t,n,X,θ=θ,limits=limits)
        val += tmp[1]
        integration_error += tmp[2]
        integration_tail += tmp[3]
        n += 1
    end
    return val, integration_error, integration_tail, Ht_tail(x,y, t=t, θ=θ, n0=N+1)[1]
end

pwHt (generic function with 1 method)

In [12]:
setprecision(128) do
    pwHt(x,y,t,5,big(0.7))
end

n = 1
tmp = Itθ_term(x, y, t, n, X, θ=θ, limits=limits) = (-1.451852135003693296983396392949034986671e-07 + 8.510239116465720866507421955791818686072e-10im, 4.998075638709847268213899407420678564514e-23, 1.310180583934037110297099873274631161073e-23)
n = 2
tmp = Itθ_term(x, y, t, n, X, θ=θ, limits=limits) = (1.451849994779719001350852986675613361725e-07 - 8.510227300988192781830930016574998722934e-10im, 1.247058973242079292382642362651630938945e-26, 4.464088297329689541291391884621508380457e-94)
n = 3
tmp = Itθ_term(x, y, t, n, X, θ=θ, limits=limits) = (2.140223948842402596847412682548830615862e-13 - 1.181547739739979415032409849755396089298e-15im, 7.617575670306368837978149535989267423257e-33, 3.866010876168112035415803894009909227927e-212)
n = 4
tmp = Itθ_term(x, y, t, n, X, θ=θ, limits=limits) = (8.827995412067709474978190701783577077257e-23 - 3.587617567334039363461959563745749604275e-25im, 9.829429901653105361982391967294793421237e-42, 1.15128433520702379601350605780050816521e-377

(-2.457043040535367641338357551201973955035e-21 + 1.270972647742896937353839064877529273678e-23im, 4.999322698444846915519861923747711876082e-23, 1.310180583934037110297099873274631161073e-23, 4.072706751498916e-49)

### Remarks

That the n=1 and n=2 terms almost cancel is partially offset by multiprecision arithmetic. The result does not inspire confidence, however. Consider,

In [19]:
setprecision(128)do
    @show I1 = Itθ_term(x,y,t,1,0.7; θ=θ, limits=fine_limits)
    @show I2 = Itθ_term(x,y,t,2,0.7; θ=θ, limits=fine_limits)
    @show I1[1]+I2[1]
    @show I3 = Itθ_term(x,y,t,3,0.7; θ=θ, limits=fine_limits)
    @show I4 = Itθ_term(x,y,t,4,0.7; θ=θ, limits=fine_limits)
    @show I5 = Itθ_term(x,y,t,5,0.7; θ=θ, limits=fine_limits)
    @show I1[1]+I2[1]+I3[1]+I4[1]+I5[1]
end
nothing

I1 = Itθ_term(x, y, t, 1, 0.7; θ=θ, limits=fine_limits) = (-1.451852136268564e-7 + 8.510239638657927e-10im, 5.517775170572531e-18, 6.446592467831734e-16)
I2 = Itθ_term(x, y, t, 2, 0.7; θ=θ, limits=fine_limits) = (1.451849994779722e-7 - 8.510227300987647e-10im, 9.94931222451599e-22, 1.2804867371224411e-62)
I1[1] + I2[1] = -2.141488842074088e-13 + 1.233767027968232e-15im
I3 = Itθ_term(x, y, t, 3, 0.7; θ=θ, limits=fine_limits) = (2.140223948842405e-13 - 1.1815477397399568e-15im, 5.011277829899066e-28, 4.694145406499207e-141)
I4 = Itθ_term(x, y, t, 4, 0.7; θ=θ, limits=fine_limits) = (8.827995412067662e-23 - 3.5876175673353375e-25im, 5.74079177660948e-37, 4.217337577724118e-251)
I5 = Itθ_term(x, y, t, 5, 0.7; θ=θ, limits=fine_limits) = (-1.58662180527884e-34 + 8.614776436687162e-37im, 3.968734138019792e-49, 1.02582354894234154993898594928236308425e-392)
I1[1] + I2[1] + I3[1] + I4[1] + I5[1] = -1.2648923488835377e-16 + 5.221928786951347e-17im


Although piecewise quadrature seems accurate, it appears that the strategy of integrating before summing falls prey to cancellation between terms in the infinite series. Testing another value of x to see if the problem is general,

In [20]:
setprecision(128) do
    pwHt(600,y,t,5,big(0.7))
end

n = 1
tmp = Itθ_term(x, y, t, n, X, θ=θ, limits=limits) = (-2.668845372347370910407490386208048623491e-09 + 1.335098172182516194850376001898283620179e-11im, 1.047787691139686792929869598745034341796e-24, 5.667392837327751496440542010865918253444e-24)
n = 2
tmp = Itθ_term(x, y, t, n, X, θ=θ, limits=limits) = (2.668840978944270687053258214379693054909e-09 - 1.335096001344772343909956128278064661162e-11im, 2.957257692333641648014102763265965405172e-28, 2.678461549684502073900192994574720105223e-91)
n = 3
tmp = Itθ_term(x, y, t, n, X, θ=θ, limits=limits) = (4.393403048369351695083541952281804231626e-15 - 2.170837736103196885439915720130832191927e-17im, 1.074030953434929311814425675365779936833e-34, 4.016421668040083559295452334119967016691e-204)
n = 4
tmp = Itθ_term(x, y, t, n, X, θ=θ, limits=limits) = (2.937612038384558836922176463067597673741e-24 - 1.370846232277657140977836604518951935961e-26im, 8.288236231769739865100892202061751552388e-44, 2.580830915276628367952621495319367892433e-36

(-4.891639049870658369587049680836146748213e-23 + 6.376897302157537995321912901291382880751e-26im, 1.048083417016323252521046302520663877118e-24, 5.667392837327751496440542010865918253444e-24, 1.6775389764304526e-50)